In [ ]:
# setting the Thesis project folder as working directory
%cd "../.."

Import packages

In [ ]:
import random
import wandb
import tensorflow as tf
import matplotlib.pyplot as plt
import matplotlib as mpl
import numpy as np
# import pandas as pd
# import pickle
import seaborn as sns
import os

from tensorflow.keras.preprocessing.image import ImageDataGenerator
# from keras_unet_collection import models
from keras_unet_collection.losses import tversky, iou_seg, focal_tversky
from skimage import io
# from PIL import Image
from skimage.measure import label
from skimage.measure import regionprops

import sys
from utility_functions.utility import *

## Setting default parameters

In [ ]:
# Random seed for reproducibility
seed = 42

random.seed(seed)
os.environ['TF_CUDNN_DETERMINISTIC'] = '1'
os.environ['PYTHONHASHSEED'] = str(seed)
np.random.seed(seed)
tf.random.set_seed(seed)
tf.compat.v1.set_random_seed(seed)

# plots sizes
plt.rcParams['figure.figsize'] = [15, 15]

# Import data and scale

In [ ]:
X, Y = download_data(project_name='Team-Artifacts-Tesi', artifact_path='img3_test_set:v1', entity='white-mad-hatters',)

In [ ]:
X.shape, Y.shape, np.max(X), np.min(X)

# Load model

In [ ]:
model = tf.keras.models.load_model("models\model-best.h5", custom_objects={'tversky': tversky})
# loaded_model. build()

In [ ]:
model.summary()

# Output image

In [ ]:
img_size = 128
# z_chan = 0

In [ ]:
# %%time
y_pred = predict_image(model, X[ :, :, 0], img_size, y_sum_tresh=1)

In [ ]:
plt.imshow(X[1500:2000, 5000:5500, 0])

In [ ]:
plt.imshow(y_pred[1500:2000, 5000:5500])
plt.show()
plt.imshow(Y[1500:2000, 5000:5500])

In [ ]:
plt.imshow(y_pred)
plt.show()
plt.imshow(Y)

2D Output image with TP, TN, FP, FN coloured

In [ ]:
# round predicted image to 0 and 1
y_pre_rd = np.around(y_pred)

# create final coloured image with same shape of the predicted one
y_final = np.full((y_pred.shape), -1, dtype=np.int8)

# assign Y
Y_new = Y#[:, :, z_chan]

# substitute tp, tn, fp, fn with a number
y_final[np.logical_and(Y_new, y_pre_rd)] = 0 # tp
y_final[np.logical_and(np.logical_not(Y_new), np.logical_not(y_pre_rd))] = 1 # tn
y_final[np.logical_and(np.logical_not(Y_new), y_pre_rd)] = 2 #fp
y_final[np.logical_and(Y_new, np.logical_not(y_pre_rd))] = 3 #fn

print(np.unique(y_final) )
tp = np.sum(y_final == 0)
tn = np.sum(y_final == 1)
fp = np.sum(y_final == 2)
fn = np.sum(y_final == 3)

# Plot the image
colors = ['green', 'blue', 'red', 'yellow']
bounds = [0,1,2,3,4]

cmap = mpl.colors.ListedColormap(colors)
norm = mpl.colors.BoundaryNorm(bounds, cmap.N)

fig = plt.figure()
ax = plt.Axes(fig, [0., 0., 1., 1.])
# ax.set_axis_off()
fig.add_axes(ax)
#plt.set_cmap(cmap)
ax.imshow(y_final, interpolation='none', cmap=cmap, norm=norm)#aspect='equal')

#plt.imshow(y_plot, interpolation='none', cmap=cmap, norm=norm)
# plt.savefig('output/composite.png', dpi = 620)
print('recall: ', f"{tp/(tp+fn):.4f}", '\n'
      'precision: ', f"{tp/(tp+fp):.4f}")
plt.show()

Tensor Output image with TP, TN, FP, FN in different slices

To check:
- check Y

In [ ]:
# round predicted image to 0 and 1
y_pre_rd = np.around(y_pred)
Y_new = Y#[:, :, Y_slice]

# create final coloured image with same shape of the predicted one
y_final = np.full((4, y_pre_rd.shape[0], y_pre_rd.shape[1]), -1, dtype=np.int8)

# substitute tp, tn, fp, fn with a number
y_final[0] = np.logical_and(Y_new, y_pre_rd).astype(int) # tp
y_final[1] = np.logical_and(np.logical_not(Y_new), np.logical_not(y_pre_rd)).astype(int) # tn
y_final[2] = np.logical_and(np.logical_not(Y_new), y_pre_rd).astype(int) #fp
y_final[3] = np.logical_and(Y_new, np.logical_not(y_pre_rd)).astype(int) #fn

print(np.unique(y_final))

In [ ]:
y_final.shape

In [ ]:
io.imsave('output/tensor.tif', np.around(y_final), photometric='minisblack')